In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [23]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

## 1. Convolution 연산, Flatten 연산, Fully Connected 레이어, Activation 연산만을 가지고 MNIST classification 수행

In [24]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 28 * 28, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = x.view(-1, 64 * 28 * 28)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [25]:
model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [26]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

In [27]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n')

In [28]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

num_epochs = 10
for epoch in range(1, num_epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.298962
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.178027
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.101750
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.121671
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.100641
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.046540
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.108582
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.044171
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.040359
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.039863

Test set: Average loss: 0.0001, Accuracy: 9807/10000 (98%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.016147
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.017861
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.040031
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.001865
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.062226
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.033769
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.023865
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.081789
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.025124
T

## 2. 1번 SimpleCNN에 Max Pooling, Average Pooling 레이어를 추가하여 MNIST Classification 수행

In [16]:
class CNNWithPooling(nn.Module):
    def __init__(self):
        super(CNNWithPooling, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.max_pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.avg_pool = nn.AvgPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.max_pool(x)
        x = F.relu(self.conv2(x))
        x = self.avg_pool(x)
        x = x.view(-1, 64 * 7 * 7) 
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNNWithPooling().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [18]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')


In [19]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.2f}%)\n')
    return accuracy


In [20]:
num_epochs = 10
for epoch in range(1, num_epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    accuracy = test(model, device, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.289431
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.317734
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.244298
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.194067
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.096510
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.064702
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.087993
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.137741
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.084311
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.045386

Test set: Average loss: 0.0001, Accuracy: 9834/10000 (98.34%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.015610
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.046879
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.036257
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.020340
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.019552
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.007244
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.060293
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.054886
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.03003

## 3. 2번 CNNWithPooling에 Pooling 연산을 제거하고 Adaptive Pooling을 적절히 활용하여 MNIST Classification 수행

In [26]:
class CNNWithAdaptivePooling(nn.Module):
    def __init__(self):
        super(CNNWithAdaptivePooling, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.adaptive_pool = nn.AdaptiveAvgPool2d((4, 4))
        self.fc1 = nn.Linear(128 * 4 * 4, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = self.adaptive_pool(x)
        x = x.view(-1, 128 * 4 * 4)  
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [36]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNNWithAdaptivePooling().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [37]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

In [38]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.2f}%)\n')
    return accuracy

In [39]:
num_epochs = 20
for epoch in range(1, num_epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    accuracy = test(model, device, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.304399
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.214516
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.159814
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.068543
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.111902
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.195829
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.037957
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.115686
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.131907
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.056372

Test set: Average loss: 0.0000, Accuracy: 9835/10000 (98.35%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.023173
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.019144
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.035261
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.007696
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.056390
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.028300
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.086048
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.026697
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.01865

In [43]:
class CNNWithAdaptivePooling(nn.Module):
    def __init__(self):
        super(CNNWithAdaptivePooling, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.adaptive_pool = nn.AdaptiveAvgPool2d((4, 4))
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.adaptive_pool = nn.AdaptiveAvgPool2d((4, 4))
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.adaptive_pool = nn.AdaptiveAvgPool2d((4, 4))
        self.fc1 = nn.Linear(128 * 4 * 4, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.adaptive_pool(x)
        x = F.relu(self.conv2(x))
        x = self.adaptive_pool(x)
        x = F.relu(self.conv3(x))
        x = self.adaptive_pool(x)
        x = x.view(-1, 128 * 4 * 4)  
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [44]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNNWithAdaptivePooling().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [45]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

In [46]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.2f}%)\n')
    return accuracy

In [47]:
num_epochs = 20
for epoch in range(1, num_epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    accuracy = test(model, device, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.306063
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.459349
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.267839
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.306150
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.105324
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.102394
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.060040
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.153130
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.149157
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.139501

Test set: Average loss: 0.0001, Accuracy: 9710/10000 (97.10%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.053973
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.034867
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.091901
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.181417
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.021572
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.019549
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.112175
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.017251
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.03862

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2
import numpy as np

In [14]:
class MNISTDataset(Dataset):
    def __init__(self, mnist_dataset, transform=None):
        self.mnist_dataset = mnist_dataset
        self.transform = transform

    def __len__(self):
        return len(self.mnist_dataset)

    def __getitem__(self, idx):
        image, label = self.mnist_dataset[idx]
        image = np.array(image)

        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']

        return image, label

In [15]:
transform = A.Compose([
    A.RandomRotate90(p=0.5),
    A.Transpose(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.5, rotate_limit=45, p=0.5),
    A.Blur(blur_limit=3, p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.Normalize((0.1307,), (0.3081,)),
    ToTensorV2()
])

In [16]:
train_dataset = datasets.MNIST('./data', train=True, download=True)
test_dataset = datasets.MNIST('./data', train=False, download=True, transform=transforms.ToTensor())

augmented_train_dataset = MNISTDataset(train_dataset, transform=transform)
train_loader = DataLoader(augmented_train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)


In [17]:
class ImprovedCNN(nn.Module):
    def __init__(self):
        super(ImprovedCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.adaptive_pool = nn.AdaptiveAvgPool2d((4, 4))
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.adaptive_pool = nn.AdaptiveAvgPool2d((4, 4))
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.adaptive_pool = nn.AdaptiveAvgPool2d((4, 4))
        self.fc1 = nn.Linear(128 * 4 * 4, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.adaptive_pool(x)
        x = F.relu(self.conv2(x))
        x = self.adaptive_pool(x)
        x = F.relu(self.conv3(x))
        x = self.adaptive_pool(x)
        x = x.view(-1, 128 * 4 * 4)  
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ImprovedCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [19]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device).float(), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')


In [20]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device).float(), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.2f}%)\n')
    return accuracy

In [21]:
num_epochs = 100
for epoch in range(1, num_epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    accuracy = test(model, device, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.302610
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.862253
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.571985
Train Epoch: 1 [19200/60000 (32%)]	Loss: 1.295449
Train Epoch: 1 [25600/60000 (43%)]	Loss: 1.299024
Train Epoch: 1 [32000/60000 (53%)]	Loss: 1.229160
Train Epoch: 1 [38400/60000 (64%)]	Loss: 1.159708
Train Epoch: 1 [44800/60000 (75%)]	Loss: 1.118928
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.881637
Train Epoch: 1 [57600/60000 (96%)]	Loss: 1.102810

Test set: Average loss: 0.0009, Accuracy: 7797/10000 (77.97%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 1.189799
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.740614
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.669652
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.696588
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.777037
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.637483
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.668330
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.662918
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.76273